# Darcy interface problem

In [1]:
import sys
sys.path.append('../../python/darcy/')
from darcy_wrapper import *
from darcy_data_example1_2D import *
import matplotlib.pyplot as plt
import numpy as np

Given $\mathbf{f}: \Omega_{1}\cup  \Omega_{2} \rightarrow \mathbb{R}^d$,  $g: \Omega_{1}\cup  \Omega_{2} \rightarrow \mathbb{R}$ , boundary data $p_B: \partial\Omega_{p} \rightarrow \mathbb{R}$, $u_B:  \partial\Omega_{\mathbf{u}} \rightarrow \mathbb{R}$ where $\partial \Omega= \partial\Omega_{p}  \cup  \partial\Omega_{\mathbf{u}}$, with  $\partial\Omega_{p}  \cap  \partial\Omega_{\mathbf{u}} = \emptyset$, a prescribed fluid pressure in the fracture $\hat{p}: \Gamma \rightarrow \mathbb{R}$, the inverse permeability $\boldsymbol{\eta}$ ($\boldsymbol{\eta}=\mathbf{K}^{-1}$ with $\mathbf{K}: \Omega_{1}\cup  \Omega_{2} \rightarrow \mathbb{R}^d$ being the permeability tensor) and $\boldsymbol{\eta}_\Gamma$ that depends on the permeability tensor in the fracture and the fracture thickness, we seek fluid velocity $\mathbf{u}: \Omega_{1}\cup  \Omega_{2} \rightarrow \mathbb{R}^d$ and pressure $p: \Omega_{1}\cup  \Omega_{2} \rightarrow \mathbb{R}$ satisfying the following Darcy interface problem: 
\begin{alignat*}{2}
  \boldsymbol{\eta} \mathbf{u} + \nabla p & = \mathbf{f} 
 && \text{ in } \Omega_1\cup\Omega_2, \\
  \text{div} \mathbf{u} & =  g
  &&\text{ in } \Omega_1\cup\Omega_2, \\
  \llbracket p \rrbracket & =  \boldsymbol{\eta}_\Gamma \{\mathbf{u}\cdot\mathbf{n}\}
  &&\text{ on } \Gamma,  \\
  \{p\} & =  \hat{p}+\xi \boldsymbol{\eta}_\Gamma \llbracket \mathbf{u}\cdot\mathbf{n} \rrbracket
  &&\text{ on } \Gamma, \\
  p & =  p_B
  &&\text{ on } \partial\Omega_{p},\\
  \mathbf{u}\cdot\mathbf{n} & =  u_B
  &&\text{ on } \partial\Omega_{\mathbf{u}}. 
\end{alignat*}
Here, the parameter $\xi\in(0,1/4]$, $\mathbf{f} \in \mathbf{L}^2({\Omega_1 \cup \Omega_2})$, $g \in L^2({\Omega_1 \cup \Omega_2})$,  $p_B \in H^{1/2}(\partial\Omega_{p})$, $u_B \in H^{-1/2}(\partial\Omega_{\mathbf{u}})$, $\hat{p} \in H^{1/2}(\Gamma)$. We assume that the permeability tensor $\mathbf{K}$ is diagonal with the smallest diagonal entry being positive and bounded away from zero.


In [2]:
g               = USER_FUNC(func_div)
p_B             = USER_FUNC(func_neumann)
p_hat           = USER_FUNC(func_phat)
fun_velocity    = USER_FUNC(func_velocity)
fun_pressure    = USER_FUNC(func_pressure)
fun_level_set   = USER_FUN_LS(func_level_set)

set_verbose(1)
darcy = DarcyCutFEM()
darcy.build_mesh(11,11 ,sq_SW, sq_SW, sq_LGTH, sq_LGTH)
darcy.init_space(fun_level_set, 'RT0')

 ---  INFO CUT MESH  --- 
 Number of subdomains          :	2
 Number of elements in Omega_0 :	174
 Number of elements in Omega_1 :	60
 Total number of elements      :	234
 Velocity space : 
 nb node    	388
 nb dof     	388
 nb element 	234
 Pressure space : 
 nb node    	234
 nb dof     	234
 nb element 	234


\begin{align}
	a(\mathbf{u},\mathbf{v}) + b(\mathbf{v},p) &= \mathcal{F} (\mathbf{v}) \text{ for all } \mathbf{v}\in \mathbf{V}_0, \\
	b(\mathbf{u},q) &= \mathcal{G} (q) \text{ for all } q\in Q
\end{align}
with 
\begin{align}
	a(\mathbf{u},\mathbf{v}) &:= (\boldsymbol{\eta} \mathbf{u},\mathbf{v})_{\Omega_1 \cup \Omega_2} +  (\boldsymbol{\eta}_{\Gamma} \{\mathbf{u}\cdot\mathbf{n}\},\{\mathbf{v}\cdot\mathbf{n}\})_{\Gamma} + (\xi\boldsymbol{\eta}_{\Gamma} [ \mathbf{u}\cdot\mathbf{n} ], [\mathbf{v}\cdot\mathbf{n}])_{\Gamma} \\
	b(\mathbf{u},q) &:= -(\nabla \cdot \mathbf{u},q)_{\Omega_1 \cup \Omega_2}  \\
	\mathcal{F} (\mathbf{v}) &:= (\mathbf{f},\mathbf{v})_{\Omega_1 \cup \Omega_2} 
	- (p_B,\mathbf{v}\cdot \mathbf{n})_{\partial \Omega_p}  - (\hat{p}, [\mathbf{v}\cdot\mathbf{n}])_{\Gamma} \\
	\mathcal{G}(q) &:= - (g,q)_{\Omega_1 \cup \Omega_2}
\end{align}


In [3]:
darcy.add_bulk_integral(g)
darcy.add_interface_integral(p_hat)
darcy.add_natural_BC(p_B)

Each element in the active mesh $\mathcal{T}_{h,i}$ is classified either as having a large intersection with the domain $\Omega_i$, or a small intersection. We say that an element $T \in \mathcal{T}_{h,i}$ has a large $\Omega_i$-intersection if
\begin{equation}
  \delta_i  \leq \frac{|T \cap \Omega_i|}{|T|}.
\end{equation}

Stabilization is applied only on internal faces of macro-elements. The stabilization terms are 

\begin{align}
  s_{\mathbf{u}}(\mathbf{u}_h,\mathbf{v}_h) &= \sum_{i=1}^2 \sum_{M_L \in \mathcal{M}_{h,i}} \sum_{F \in \mathcal{F}(M_L)} \sum_{j=0}^{\hat{k}+1} \tau_{u} h^{2j + 1} ([D^j_{\mathbf{n}_F} \mathbf{u}_{h,i} ], [D^j_{\mathbf{n}_F} \mathbf{v}_{h,i}])_{F}, \\
  s_{b}(\mathbf{u}_h,q_h) &= \sum_{i=1}^2 \sum_{M_L \in \mathcal{M}_{h,i}} \sum_{F \in \mathcal{F}(M_L)}  \sum_{j=0}^{\hat{k}} \tau_{b} h^{2j + \gamma} ( [D^j (\nabla  \mathbf{u}_{h,i})], [D^j q_{h,i}] )_{F}. 
\end{align}

In [4]:
tau_u = 0.1
tau_b = 0.1
delta_i = 0.25
darcy.set_stabilization_penalty(tau_u, tau_b)
darcy.add_macro_stabilization(delta_i)

 ---  INFO MACRO ELEMENT  --- 
 Tolerance small element  :	0.00125
 Small element in Omega 1 :	4
 Small element in Omega 2 :	20


In [5]:


darcy.solve()

In [8]:


error_divu_L2 = darcy.L2error_div(g)
error_u_L2    = darcy.L2error_vel(fun_velocity)
error_p_L2    = darcy.L2error_pressure(fun_pressure)

print(" L2 error p      :", error_p_L2)
print(" L2 error u      :", error_u_L2)
print(" L2 error div(u) :", error_divu_L2)

 L2 error p      : 0.16080801821777166
 L2 error u      : 0.030387743609381273
 L2 error div(u) : 5.3811358199658576e-14
